In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(5603, 12)

In [4]:
len(df[df.tiling_compatible == True]) 

181

In [5]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   2052
1  cant_extract_variables   1949
2        no_granule_found    971
3  tile_generation_failed    402
4                    None    181
5                 timeout     48


In [13]:
df[(df.incompatible_reason == "cant_extract_variables") & (df['data_variables'].isna())]

,collection_concept_id,concept_id,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables
64,C2532426483-ORNL_CLOUD,G2533273830-ORNL_CLOUD,s3://ornl-cumulus-prod-protected/daymet/Daymet...,None,None,nc,False,cant_extract_variables,Can't extract variables,None,None,None
72,C3294057315-ASF,G3681458914-ASF,s3://asf-cumulus-prod-opera-products/OPERA_L3_...,None,netCDF-4,nc,False,cant_extract_variables,Can't extract variables,None,None,None
98,C1235338554-GES_DISC,G1241497587-GES_DISC,s3://gesdisc-cumulus-prod-protected/WAOB/LPRM_...,None,netCDF,nc,False,cant_extract_variables,Can't extract variables,None,None,None
104,C1235338549-GES_DISC,G1241492468-GES_DISC,s3://gesdisc-cumulus-prod-protected/WAOB/LPRM_...,None,netCDF,nc,False,cant_extract_variables,Can't extract variables,None,None,None
105,C2237824918-ORNL_CLOUD,G2613728971-ORNL_CLOUD,s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A...,None,None,h5,False,cant_extract_variables,Can't extract variables,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
5324,C1979728494-GHRC_DAAC,G1979728737-GHRC_DAAC,s3://ghrcw-protected/gpmsimorblpvex__1/wrfout_...,None,None,nc,False,cant_extract_variables,Can't extract variables,None,None,None
5325,C1979728927-GHRC_DAAC,G1979729085-GHRC_DAAC,s3://ghrcw-protected/gpmsimorbmc3e__1/wrfout_d...,None,None,nc,False,cant_extract_variables,Can't extract variables,None,None,None
5326,C1979729803-GHRC_DAAC,G1979730005-GHRC_DAAC,s3://ghrcw-protected/gpmsimorbtwpice__1/GCEBN3...,None,None,nc,False,cant_extract_variables,Can't extract variables,None,None,None
5330,C1979830182-GHRC_DAAC,G1979830333-GHRC_DAAC,s3://ghrcw-protected/gpmvertixgcpex__1/vertix_...,None,None,nc,False,cant_extract_variables,Can't extract variables,None,None,None


In [49]:
import numpy as np

# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and x.size > 0)
)]
subset_df

,collection_concept_id,concept_id,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables
2,C2069247178-GES_DISC,G2071455506-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, SWdown, LWdown, SWnet, LWnet, Qle,..."
4,C2033151148-GES_DISC,G2033667526-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, Tair, Qair, PSurf, Wind_E, Wind_N,..."
6,C2069247359-GES_DISC,G2076482666-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, SWdown, LWdown, SWnet, LWnet, Qle,..."
7,C2033167496-GES_DISC,G2035054839-GES_DISC,s3://gesdisc-cumulus-prod-protected/NLDAS/NLDA...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,"[time_bnds, SWdown, Rainf, CRainf, ACond, Tair..."
10,C2021957295-LPCLOUD,G2705887278-LPCLOUD,s3://lp-prod-protected/HLSS30.020/HLS.S30.T33K...,rasterio,COG,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[NIR_Narrow]
...,...,...,...,...,...,...,...,...,...,...,...,...
2910,C1652491691-GES_DISC,G1652768767-GES_DISC,s3://gesdisc-cumulus-prod-protected/CMS/CMSLak...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[app]
2912,C1652491711-GES_DISC,G1652768756-GES_DISC,s3://gesdisc-cumulus-prod-protected/CMS/CMSLak...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[app]
2914,C1652491837-GES_DISC,G1652768721-GES_DISC,s3://gesdisc-cumulus-prod-protected/CMS/CMSLak...,xarray,netCDF,nc,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[app]
3222,C3271567778-NSIDC_CPRD,G3271605286-NSIDC_CPRD,s3://nsidc-cumulus-prod-protected/SNOWEX/SNEX2...,rasterio,None,tif,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,None,[None]


In [11]:
!aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
    s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
